In [1]:
import os
from IPython.display import clear_output


notebook_dir = "/home/balabaevvl/courses/project/FITS/src/"
os.chdir(notebook_dir)

GPUs = [
    "GPU-e83bd31b-fcb9-b8de-f617-2d717619413b",
    "GPU-5a9b7750-9f85-49a5-3aae-fe07b1b7661d",
    "GPU-fe2d8dfd-06f2-a5c4-a7fd-4a5f23947005",
    "GPU-0c320096-21ee-4060-8731-826ca2febfab",
    "GPU-baef952c-6609-aace-3b78-e4e07788d5de",
    "GPU-3979d65b-c238-4e9c-0c1c-1aa3f05c56a1",
    "GPU-6c76a2c5-5375-aa06-11d4-0fddfac30e91",
]
os.environ["CUDA_VISIBLE_DEVICES"] = f"{GPUs[1]}"

In [2]:
from fits.dataframes.download import DownloadDatasetAirQuality

DownloadDatasetAirQuality()

In [3]:
# import pandas as pd
# from fits.config import DatasetsPaths

# df = pd.read_csv(
#     DatasetsPaths.pm25.value,
#     index_col="datetime",
#     parse_dates=True,
# )

# df.head()

In [4]:
# from fits.dataframes.dataset import ModelMode, DatasetAirQuality

# dataset = DatasetAirQuality(ModelMode.train)

# for sample in dataset:
#     break
# sample

In [5]:
import torch
from fits.modelling.framework import Train, Evaluate
from fits.dataframes.dataset import DatasetAirQuality
from fits.dataframes.dataloader import ForecastingDataLoader
from fits.modelling.comparison import (
    CalculateParams,
    ReadMetrics,
    VisualizeForecastSample,
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
train_loader, valid_loader, test_loader = ForecastingDataLoader(
    DatasetAirQuality, batch_size=128  #!
)
normalization_stats = train_loader.dataset.normalization_stats

EPOCHS = 300
NSAMPLE = 10

# VAR

In [7]:
from fits.modelling.VAR.model import SeasonalVAR, VARConfig

var = SeasonalVAR(VARConfig(lag_order=12)).to(device)

print(f"Total params: {CalculateParams(var)}")

Total params: 16452


In [ ]:
Train(var, train_loader, valid_loader, epochs=EPOCHS)

In [ ]:
Evaluate(var, test_loader, normalization_stats, nsample=NSAMPLE)

In [8]:
eval_foldername = "SeasonalVAR_20260105_145357"

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

RMSE: 46.389682908606076
MAE: 31.253435324430423
CRPS: 0.39001720830013875
CRPS_sum: 0.32056725652594315


{'rmse': 46.389682908606076,
 'mae': 31.253435324430423,
 'crps': 0.39001720830013875,
 'crps_sum': 0.32056725652594315}

# CSDI

In [9]:
from fits.modelling.CSDI.adapter import CSDIAdapter


csdi = CSDIAdapter().to(device)
print(f"Total params: {CalculateParams(csdi)}")

Total params: 602113


/home/balabaevvl/courses/venv311/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
Train(csdi, train_loader, valid_loader, epochs=EPOCHS)

In [ ]:
# model_foldername = "CSDIAdapter_20260102_161116"

# state = torch.load(
#     f"../data/models/training/{model_foldername}/model.pth",
#     map_location=device,
# )

# csdi.load_state_dict(state)
# csdi.to(device)

# clear_output()

In [ ]:
Evaluate(csdi, test_loader, normalization_stats, nsample=NSAMPLE)

In [10]:
eval_foldername = "CSDIAdapter_20260105_162405"

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

RMSE: 36.53446938146174
MAE: 21.72300644442995
CRPS: 0.22033239665784335
CRPS_sum: 0.20117890207391037


{'rmse': 36.53446938146174,
 'mae': 21.72300644442995,
 'crps': 0.22033239665784335,
 'crps_sum': 0.20117890207391037}

In [ ]:
import gc, torch

del csdi
gc.collect()
torch.cuda.empty_cache()

# DiffusionTS

In [11]:
from fits.modelling.DiffusionTS.adapter import DiffusionTSAdapter, DiffusionTSConfig


diffusionts = DiffusionTSAdapter().to(device)
print(f"Total params: {CalculateParams(diffusionts)}")

Total params: 1389172


In [ ]:
Train(diffusionts, train_loader, valid_loader, epochs=EPOCHS)
# 1.6 - min

In [ ]:
# model_foldername = "DiffusionTSAdapter_20260104_211002"

# state = torch.load(
#     f"../data/models/training/{model_foldername}/model.pth",

#     map_location=device,
# )

# diffusionts.load_state_dict(state)
# diffusionts.to(device)

# clear_output()

In [ ]:
Evaluate(diffusionts, test_loader, normalization_stats, nsample=NSAMPLE)
# TODO: epochs 100->300; enc=4 dec=3; timesteps=1000
# TODO: try lr=1e-4 -> works the same
# TODO: langevin_coef=0.0

In [12]:
# eval_foldername = "DiffusionTSAdapter_20260105_173500"
# eval_foldername = "DiffusionTSAdapter_500_500"
eval_foldername = "DiffusionTSAdapter_500_500_upd_langevein"
# eval_foldername = "DiffusionTSAdapter_500_500_no_langevein"

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

RMSE: 43.83098781188662
MAE: 29.966135980881674
CRPS: 0.32084613097341436
CRPS_sum: 0.2489513597990337


{'rmse': 43.83098781188662,
 'mae': 29.966135980881674,
 'crps': 0.32084613097341436,
 'crps_sum': 0.2489513597990337}

# FM-TS

In [ ]:
from fits.modelling.FMTS.adapter import FMTSAdapter


fmts = FMTSAdapter().to(device)
print(f"Total params: {CalculateParams(fmts)}")

In [ ]:
Train(fmts, train_loader, valid_loader, epochs=EPOCHS)

In [ ]:
Evaluate(fmts, test_loader, normalization_stats, nsample=NSAMPLE)

In [ ]:
eval_foldername = "FMTSAdapter_20260104_121131"

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

In [ ]:
# TODO: Comparison of truth VS forecasted in:
# PSA plot
# t-SNE plot
# Data Density